The decentralizedOI described in the paper

In [23]:
import numpy as np

def push_sum(B, K_i, num_iterations=100):
    """
    Decentralized Push-Sum Algorithm for computing the sum of distributed matrices.
    
    B: Stochastic transition matrix (Markov transition matrix).
    K_i: Local matrices computed at each node, representing V_i^T V_i.
    num_iterations: Number of iterations to approximate the sum.
    
    Returns:
    - Approximated sum of K(i) across all nodes.
    """
    n, k, _ = K_i.shape  # the size of the matrix K_i is n x k x k 
    S = K_i.copy()  # S_i = K_i for all nodes
    w = np.zeros(n)  
    w[0] = 1  # Set initial weight for the first node to 1, all others remain 0

    for _ in range(num_iterations):
        S_new = np.zeros((n, k, k))  
        w_new = np.zeros(n)  

        for i in range(n):  # Iterate through each node
            for j in range(n):  
                if B[i, j] > 0:  
                    S_new[i] += B[i, j] * S[j]  
                    w_new[i] += B[i, j] * w[j]  

        S = S_new  # Update sum matrix
        w = w_new  # Update weight vector
        '''accutually, it should stopped when w is evenly dsitributed, but for convenience, I just do enough iterations'''


    return S / w[:, None, None]  # Normalize to obtain the final result


In [24]:
import networkx as nx
from scipy.linalg import cholesky
import numpy as np

def decentralized_OI(G, k, num_iterations=100):
    """
    Decentralized Orthogonal Iteration (DecentralizedOI) Algorithm.
    
    This algorithm approximates the top k eigenvectors of the adjacency matrix A
    in a decentralized manner using local computations and Push-Sum aggregation.
    
    Parameters:
    G :
        The input undirected graph where nodes communicate with their neighbors.
    k : 
        The number of eigenvectors to compute.
    num_iterations : 
        The number of iterations for convergence.

    Returns:
    Q : ndarray (n, k)
        The computed k eigenvectors (each node stores its local contribution).
    """
    
    A = nx.to_numpy_array(G)
    n = A.shape[0]  # Number of nodes

    # Compute the stochastic Markov matrix B (for Push-Sum)
    degrees = np.sum(A, axis=1, keepdims=True)  # Compute node degrees
    B = np.nan_to_num(A / degrees)  # Normalize A to create B (avoid division by zero)

    # Randomly initialize Q 
    Q = np.random.rand(n, k)

    for _ in range(num_iterations):
        # Compute neighbor-weighted sum V = A @ Q
        V = A @ Q  

        # Each node computes its local K(i) = V_i^T V_i
        K_i = np.array([Vi.reshape(-1, 1) @ Vi.reshape(1, -1) for Vi in V])

        # Compute decentralized approximation of K using Push-Sum
        K = push_sum(B, K_i)  

        # print("this is K: ", K) # check if each node has the same K debugging

        # Compute Cholesky decomposition K = R^T R
        R = np.array([cholesky(K[i]) for i in range(n)])

        # S Normalize Q using R^-1
        Q = np.array([V[i] @ np.linalg.inv(R[i]) for i in range(n)])

    return Q


In [ ]:
G = nx.Graph()
edges = [(0, 1), (0, 2), (1, 2), (1, 3), (3, 4), (2, 4)]
G.add_edges_from(edges)


k = 3
Q = decentralized_OI(G, k)

print("Computed eigenvectors using DecentralizedOI:\n", Q)


this is K:  [[[13.35356806  6.61156972  8.00858407]
  [ 6.61156972  5.15736374  4.18000891]
  [ 8.00858407  4.18000891  6.01275084]]

 [[13.35356808  6.61156968  8.00858408]
  [ 6.61156968  5.15736369  4.18000888]
  [ 8.00858408  4.18000888  6.01275085]]

 [[13.35356803  6.61156975  8.00858405]
  [ 6.61156975  5.15736378  4.18000893]
  [ 8.00858405  4.18000893  6.01275083]]

 [[13.35356802  6.61156977  8.00858404]
  [ 6.61156977  5.15736381  4.18000895]
  [ 8.00858404  4.18000895  6.01275082]]

 [[13.35356809  6.61156966  8.00858409]
  [ 6.61156966  5.15736366  4.18000887]
  [ 8.00858409  4.18000887  6.01275086]]]
this is K:  [[[ 6.45489181  0.15477799 -0.9793775 ]
  [ 0.15477799  3.89662075 -0.52749198]
  [-0.9793775  -0.52749198  1.51488573]]

 [[ 6.4548918   0.15477804 -0.97937751]
  [ 0.15477804  3.89662076 -0.52749199]
  [-0.97937751 -0.52749199  1.51488574]]

 [[ 6.45489182  0.15477794 -0.97937749]
  [ 0.15477794  3.89662074 -0.52749197]
  [-0.97937749 -0.52749197  1.51488573]]

